In [ ]:
import os
import pandas as pd
import polars as pl
import excellaint as ea

test_file = "/Users/ct6g18/Python/ExcellAint/test/test_data/2_digit_yr.xlsx"

df = pd.read_excel(test_file)

df["pd_read_dates"] = pd.to_datetime(df["mangled_dates"])

df.iloc[280:295]

It seems to me like the best way to tackle this problem is to first split the string using the datetime_separator. This then gives us separate date and time columns. 

I'm going to ignore anything subsecond for now, because I'd rather focus on the dates, rather than the times - and I think that the dates are more likely to be the source of the error.

In [ ]:
print(ea.config)

In [ ]:
colnames_init = df.columns.tolist()[:2]

In [ ]:

def split_datetimes(df : pl.DataFrame | pd.DataFrame
                   ,datetime_col : str = "mangled_dates"
                   ,datetime_sep : str = ea.config.datetime_sep
                   ) -> pl.DataFrame:
    """
    Type union between pl.Dataframe and pd.DataFrame should be removed.

    To be honest, I haven't got a particularly good idea as to how this works. I
    copied it from stackoverflow 
    (https://stackoverflow.com/questions/73699500/python-polars-split-string-column-into-many-columns-by-delimiter)
    and it seems to do what I wanted.

    """

    if isinstance(df, pl.DataFrame):
        pass
    elif isinstance(df, pd.DataFrame):
        df = pl.from_pandas(df)
    else:
        raise ValueError("df must be a pandas or polars DataFrame")

    return (
        df
        .with_columns(
            pl.col("mangled_dates").str.split(ea.config.datetime_sep)
                                    .alias("[date,time]"))
        .explode("[date,time]")
        .with_columns(
            ("string_" + pl.arange(0, pl.len()).cast(pl.Utf8).str.zfill(2))
            .over("row_id")
            .alias("col_nm")
        )
        .pivot(
            index=['row_id', 'mangled_dates'],
            values='[date,time]',
            columns='col_nm',
        )
        .rename(
            {
                "string_00": "date_str",
                "string_01": "time_str"
            }
        )
    )


def convert_time(df : pl.DataFrame) -> pl.DataFrame:
    """
    Takes the time column and converts it to a time object.

    This is somewhat less ambiguous than splitting dates up, so I'm going to 
    keep it simple for now so we can resolve the main issue
    """
    return (
        df.with_columns(
            pl.col("time_str")
            .str
            .to_time(f"%H{ea.config.time_sep}%M")
            .alias("Time")
        )
    )

df = split_datetimes(df)

df = convert_time(df)

df

In [ ]:
ea.config.date_sep = "/"
# This is just a retread of the above, except now using the datetime splitter
def split_dates(df : pl.DataFrame | pd.DataFrame
               ,datetime_col : str = "mangled_dates"
               ,datetime_sep : str = ea.config.datetime_sep
               ) -> pl.DataFrame:
    """
    Type union between pl.Dataframe and pd.DataFrame should be removed.

    To be honest, I haven't got a particularly good idea as to how this works. I
    copied it from stackoverflow 
    (https://stackoverflow.com/questions/73699500/python-polars-split-string-column-into-many-columns-by-delimiter)
    and it seems to do what I wanted.

    """

    if isinstance(df, pl.DataFrame):
        pass
    elif isinstance(df, pd.DataFrame):
        df = pl.from_pandas(df)
    else:
        raise ValueError("df must be a pandas or polars DataFrame")

    return (
        df
        .with_columns(
            pl.col(datetime_col).str.split(ea.config.date_sep)
                                    .alias("[date]"))
        .explode("[date]")
        .with_columns(
            (pl.arange(0, pl.len()))
            .over("row_id")
            .alias("col_nm")
        )
        .pivot(
            index=['row_id', 'mangled_dates','Time'],
# We can make this more robust in the future by working out all the columns other 
# than the column we want to split
            values='[date]',
            columns='col_nm',
        )
    )   

df = split_dates(df,datetime_col="date_str")


### Heres the plan:

From the above, we can clearly see that 'string_00' is a days column, 'string_01' is a months column, and 'string_02' is a years column.

What heuristics am I using to do this? 

- I can see that the dates are all in the range 1-31. 
- I can see that the months are all in the range 1-12.
- Years are a bit secondary, because its not obvious. I'm going to use the heuristic that for years with 4 digits, the first two digits are 19 or 20. For years with 2 digits, the year is in the range 0-99. If the year is 2 digits and less than 24, I'm going to assume that the year is 20xx. If the year is 2 digits and greater than 24, I'm going to assume that the year is 19xx. This is a bit of a guess, but I think it's a reasonable one.

So we first need to get some statistics on our remaining columns to split, and then we need to assign a level of confidence to which are the days, months, and years columns. 

Maaaybe we could come back and use some machine learning stuff for this, but for now I think we can do it manually.

In [ ]:
import polars.selectors as cs
import warnings

cols_to_process = [col for col in df.select(cs.by_dtype(pl.Utf8)).columns if col not in colnames_init]

for col in cols_to_process:
    print(col)

# Now we need to get some of the relevant statistics on our columns.

# First thing to do is get the max character width of them all
processing_df = df.select(*cols_to_process)

max_chars_dict = processing_df.with_columns(
    [pl.col(colname).str.len_chars().max() for colname in cols_to_process]
).max().to_dict(as_series=False)

max_chars_dict = {
    key : val[0] if val else None for key,val in max_chars_dict.items()
}


max_val_dict = processing_df.with_columns(
    [pl.col(colname).cast(pl.Int32).max() for colname in cols_to_process]
).max().to_dict(as_series=False)

max_val_dict = {
    key : val[0] if val else None for key,val in max_val_dict.items()
}

def assign_datetype(cols_to_process : list[str]
                   ,max_chars_dict : dict[str,int]
                   ,max_val_dict : dict[str,int]
                   ) -> dict[str,str]:
    """
    This function takes the columns to process, the maximum number of characters
    in each column and the maximum value in each column and returns a dictionary
    of the `date data` types of each column:
        - Year 
        - Month
        - Day
    """
    available_date_data_types = { 
        "Year",
        "Month",
        "Day",
    }

    mappings = {
        "Year" : None,
        "Month" : None,
        "Day" : None,
    }

    for key, val in max_chars_dict.items():
        if val == 4:
            mappings["Year"] = int(key)
            available_date_data_types.remove("Year")

            if not ea.config.allow_monthfirst:
                mappings["Day"] = 2 - mappings["Year"]
                mappings["Month"] = 1

                available_date_data_types.remove("Day")
                available_date_data_types.remove("Month")
                break
            
            else:
                raise NotImplementedError("Only an instance with a four digit year is supported at the moment.")


    if len(available_date_data_types) > 0:
        raise AssertionError(f"Could not assign all date data types. Remaining: {available_date_data_types}. Please check the data.")

    mappings = {str(val) : key for key, val in mappings.items()}

    warnings.warn("Did not use the following arguments: {max_val_dict}, {cols_to_process}."
                  " We probably need to make this function a bit smarter."
                  ,stacklevel=2)

    return mappings
        
mappings = assign_datetype(cols_to_process,max_chars_dict,max_val_dict)

df = df.rename(mappings)

df = df.with_columns(
    pl.col("Day").cast(pl.Int32),
    pl.col("Month").cast(pl.Int32),
)


In [ ]:

def year_to_int(df : pl.DataFrame
                ,year_col : str = "Year"
                ) -> pl.DataFrame:
    """
    This gets a bit stupid, because we need to generate a new column which is a
    boolean: is the year column four characters. If so, we can cast it to an int.
    If not, we want to check if the two characters we have are less than or equal 
    to the current year. If so, we prepend "20". If not, we prepend "19".
    """

    # Spoof the current year

    CURRENT_YEAR = 24

    df = df.with_columns(
        pl.col("Year").str.len_chars().alias("year_len")
    )

    df = df.with_columns(
        pl.when(pl.col("year_len") == 4)
          .then(pl.col("Year").cast(pl.Int32))
          .otherwise(
            pl.col("Year")
          ).alias("Year")
    )

    df = df.with_columns(
        pl.when(pl.col("year_len") == 2)
        .then(
            pl.when(pl.col("Year").cast(pl.Int32) <= CURRENT_YEAR)
            .then(pl.lit("20"))
            .otherwise(pl.lit("19"))
           )
        .otherwise(pl.lit(""))
        .alias("prepend_col")
    )

    df = df.with_columns(
        (pl.col("prepend_col") + pl.col("Year").alias("Year"))
        .cast(pl.Int32)
        .alias("Year")
    )

    df = df.drop("year_len","prepend_col")

    return df.with_columns(
        pl.col(year_col).cast(pl.Int32)
    )


df = year_to_int(df)

df

In [ ]:
def combine_date_cols(df : pl.DataFrame) -> pl.DataFrame:
    """
    This function takes a dataframe with date columns and a time column and 
    combines them into a datetime column. Since the names are all set by the 
    previous functions we can keep things really constrained

    """

    df = df.with_columns(
        pl.date(
            pl.col("Year"),
            pl.col("Month"),
            pl.col("Day"),
        ).alias("Date")
        )

    return df

def combine_datetime_cols(df : pl.DataFrame) -> pl.DataFrame:
    """
    Combine our date and our time column
    """

    df = df.with_columns(
        pl.datetime(
            pl.col("Date").dt.year(),
            pl.col("Date").dt.month(),
            pl.col("Date").dt.day(),
            pl.col("Time").dt.hour(),
            pl.col("Time").dt.minute(),
            pl.col("Time").dt.second(),
        ).alias("Datetime")
        )

    df = df.drop("Date","Time","Year","Month","Day")

    return df

df = combine_date_cols(df)
df = combine_datetime_cols(df)

df


In [ ]:
df.to_pandas().plot(y="row_id")

- Long story short - it works in this specific instance. Now we just need to pull out the logic and we can get to business.


- After that - refactoring and speeding things up.

In [1]:
import excellaint as ea
import pandas as pd

ea.config.date_sep = "/"
ea.config.datetime_sep = " "


test_file = "/Users/ct6g18/Python/ExcellAint/test/test_data/2_digit_yr.xlsx"

test_df = pd.read_excel(test_file)

print(test_df)


          mangled_dates  row_id
0        01/01/20 12:00       0
1        01/01/20 01:00       1
2        01/01/20 02:00       2
3        01/01/20 03:00       3
4        01/01/20 04:00       4
...                 ...     ...
17539  31/12/2021 19:00   17539
17540  31/12/2021 20:00   17540
17541  31/12/2021 21:00   17541
17542  31/12/2021 22:00   17542
17543  31/12/2021 23:00   17543

[17544 rows x 2 columns]


In [2]:
%%timeit 
ea.parse_datetime_column(test_df,"mangled_dates")

/Users/ct6g18/Python/ExcellAint/src/excellaint/excellaint.py:80: UserWarning: Did not use the following arguments: {max_val_dict}, {cols_to_process}. We probably need to make this function a bit smarter.
  mappings = assign_datetype(config


63.5 ms ± 2.19 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
